In [13]:
from cobra.io import read_sbml_model
from cobra import Reaction, Metabolite
model = read_sbml_model('iML1515.xml')

In [2]:
model.compartments

{'c': 'cytosol', 'e': 'extracellular space', 'p': 'periplasm'}

In [3]:
for reaction in model.metabolites.gam_c.reactions:
    print(reaction.id, reaction, reaction.name)

MC6PH MC6PH: h2o_c + mchtbs6p_c --> acgam6p_c + gam_c Monoacetylchitobiose-6-phosphate hydrolase


In [4]:
for reaction in model.metabolites.chtbs_e.reactions:
    print(reaction.id, reaction, reaction.name)

EX_chtbs_e EX_chtbs_e: chtbs_e -->  Chitobiose exchange
CHTBStex CHTBStex: chtbs_e <=> chtbs_p Chitobiose transport via diffusion (extracellular to periplasm)


In [5]:
print(model.objective)

Maximize
1.0*BIOMASS_Ec_iML1515_core_75p37M - 1.0*BIOMASS_Ec_iML1515_core_75p37M_reverse_35685


In [6]:
model.objective = model.reactions.MC6PH

In [7]:
model.optimize().fluxes["MC6PH"]

0.0

In [8]:
model.medium

{'EX_pi_e': 1000.0,
 'EX_co2_e': 1000.0,
 'EX_fe3_e': 1000.0,
 'EX_h_e': 1000.0,
 'EX_mn2_e': 1000.0,
 'EX_fe2_e': 1000.0,
 'EX_glc__D_e': 10.0,
 'EX_zn2_e': 1000.0,
 'EX_mg2_e': 1000.0,
 'EX_ca2_e': 1000.0,
 'EX_ni2_e': 1000.0,
 'EX_cu2_e': 1000.0,
 'EX_sel_e': 1000.0,
 'EX_cobalt2_e': 1000.0,
 'EX_h2o_e': 1000.0,
 'EX_mobd_e': 1000.0,
 'EX_so4_e': 1000.0,
 'EX_nh4_e': 1000.0,
 'EX_k_e': 1000.0,
 'EX_na1_e': 1000.0,
 'EX_cl_e': 1000.0,
 'EX_o2_e': 1000.0,
 'EX_tungs_e': 1000.0,
 'EX_slnt_e': 1000.0}

In [9]:
model.reactions.EX_chtbs_e.lower_bound = -1000.0
model.reactions.EX_chtbs_e.upper_bound = 1000.0

In [10]:
model.reactions.EX_chtbs_e    

Reaction identifier,EX_chtbs_e
Name,Chitobiose exchange
Memory address,0x18d962a3b80
Stoichiometry,"chtbs_e <=> N,N'-diacetylchitobiose <=>"
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [15]:
medium = model.medium
with model:
    medium['EX_chtbs_e'] = 0
    # gene integration
    reaction = Reaction("asdf")
    reaction.name = "name of reaction"
    reaction.subsystem = "some metabolsim"
    reaction.lower_bound = -1000
    reaction.upper_bound = 1000

    reaction.add_metabolites({
        model.metabolites.get_by_id("glcn_c"): -1.0,
        model.metabolites.get_by_id("gln__L_c"): -1.0,
        model.metabolites.get_by_id("gam_c"): 1.0,
        model.metabolites.get_by_id("glu__L_c"): 1.0
    })
    model.add_reactions([reaction])
    model.medium = medium
    model.objective = model.reactions.asdf
    model.add_boundary(model.metabolites.gam_c, type='demand')
    solution = model.optimize()
    print(solution.fluxes[solution.fluxes != 0])
    print(solution.fluxes["asdf"])

CS          7.882105e-01
ICDHyr      7.882105e-01
EX_pi_e    -6.217249e-15
PYK        -2.168741e-15
EX_co2_e    2.364632e+00
                ...     
PItpp      -6.217249e-15
HPYRP       7.521028e-16
ACOAD1f    -4.440892e-17
asdf        9.605895e+00
DM_gam_c    9.605895e+00
Name: fluxes, Length: 97, dtype: float64
9.605894736842103
